In [15]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell

def markdown_to_jupyter(markdown_text):
    nb = new_notebook()
    lines = markdown_text.split('\n')
    current_cell = []
    in_code_block = False
    
    for line in lines:
        if line.startswith('```'):
            if in_code_block:
                # End of code block
                nb.cells.append(new_code_cell('\n'.join(current_cell)))
                current_cell = []
                in_code_block = False
            else:
                # Start of code block
                if current_cell:
                    nb.cells.append(new_markdown_cell('\n'.join(current_cell)))
                    current_cell = []
                in_code_block = True
        else:
            current_cell.append(line)
    
    # Add any remaining content as a markdown cell
    if current_cell:
        nb.cells.append(new_markdown_cell('\n'.join(current_cell)))
    
    return nb

# Example usage
markdown_text = r''''''

In [16]:
notebook = markdown_to_jupyter(markdown_text)
with open('output_notebook.ipynb', 'w') as f:
    nbformat.write(notebook, f)